# Create an Azure AI Content Safety (AACS) enabled text-to-image batch endpoint (Preview)
### This notebook is under preview.

### Steps to create an __AACS__ enabled __text-to-image__ batch endpoint
1. Create a __AACS__ enabled text-to-image batch endpoint with a custom [score_batch.py](./aacs-scoring-files/score/score_batch.py) script. This will integrate the batch endpoint with the AACS resource to moderate the response from the __text-to-image__ model and the request from the user.
2. Create a new __AACS__ enabled __text-to-image__ batch endpoint with a custom [score_batch.py](./aacs-scoring-files/score/score_batch.py) which will integrate with the __AACS__ resource to moderate the response from the __text-to-image__ model and the request from the user. To make the custom [score_batch.py](./aacs-scoring-files/score/score_batch.py) to successfully authenticated to the __AACS__ resource, use __Environment variable__ to pass the access key of the __AACS__ resource to the custom [score_batch.py](./aacs-scoring-files/score/score_batch.py) via environment variable. The custom [score_batch.py](./aacs-scoring-files/score/score_batch.py) can use the key directly to access the AACS resource. This option is less secure, if someone in your org has access to the endpoint, he/she can get the access key from the environment variable and use it to access the AACS resource.

### Task

`text-to-image` tasks generates image as output based on text prompt given in input.
 
### Model
Models that can perform the `text-to-image` task are tagged with `text-to-image`. We will use the `runwayml-stable-diffusion-v1-5` model in this notebook. If you opened this notebook from a specific model card, remember to replace the specific model name.

### Outline
1. Setup pre-requisties
2. Create AACS resource
3. Create AACS enabled text-to-image batch endpoint
4. Prepare data for inference - using a folder of csv files with text prompts
5. Test the endpoint - using csv files
6. Clean up resources - delete the endpoint


### 1. Setup pre-requisties

* Check List
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry

> [x] The identity you are using to execute this notebook(yourself or your VM) need to have the __Contributor__ role on the resource group where the AML Workspace your specified is located, because this notebook will create an AACS resource using that identity.

In [ ]:
# Install the required packages
%pip install azure-identity==1.13.0
%pip install azure-mgmt-cognitiveservices==13.4.0
%pip install azure-ai-ml>=1.23.1
%pip install azure-mgmt-msi==7.0.0
%pip install azure-mgmt-authorization==3.0.0

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient
from azure.mgmt.cognitiveservices.models import Account, Sku, AccountProperties
from IPython.core.display import display, HTML
from azure.ai.ml import Input
from azure.ai.ml.entities import (
    ModelBatchDeployment,
    ModelBatchDeploymentSettings,
    AmlCompute,
    Data,
    BuildContext,
    BatchRetrySettings,
    CodeConfiguration,
    Environment,
)

In [ ]:
import time

timestamp = str(time.time()).split(".")[0]

# The public registry name contains text-to-image models
registry_name = "azureml"

endpoint_name = f"text-to-image-test-ep-{timestamp}"  # Replace with your endpoint name,
deployment_name = "default"  # Replace with your deployment name, lower case only!!!
sku_name = "STANDARD_NC4AS_T4_V3"  # Name of the sku(compute instance type)

environment_name = "text-to-image-model-env"  # Replace with your environment name
compute_name = "gpu-compute"  # Replace with your compute name

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [ ]:
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # enter details of your AML workspace
    subscription_id = "subscription_id"
    resource_group = "resource_group"
    workspace = "workspace"

    # get a handle to the workspace
    ml_client = MLClient(
        credential,
        subscription_id,
        resource_group,
        workspace,
        logging_enable=True,
    )


subscription_id = ml_client.subscription_id
resource_group = ml_client.resource_group_name
workspace = ml_client.workspace_name

reg_client = MLClient(
    credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    registry_name=registry_name,
)


print(f"Connected to workspace {workspace}")

### 2. Create AACS

#### 2.1 Assign variables for Azure Content Safety
Currently, AACS is available in a limited set of regions:

__NOTE__: before you choose the region to deploy the AACS, please be aware that your data will be transferred to the region you choose and by selecting a region outside your current location, you may be allowing the transmission of your data to regions outside your jurisdiction. It is important to note that data protection and privacy laws may vary between jurisdictions. Before proceeding, we strongly advise you to familiarize yourself with the local laws and regulations governing data transfer and ensure that you are legally permitted to transmit your data to an overseas location for processing. By continuing with the selection of a different region, you acknowledge that you have understood and accepted any potential risks associated with such data transmission. Please proceed with caution.

In [ ]:
acs_client = CognitiveServicesManagementClient(credential, subscription_id)


# settings for the AACS resource
# we will choose existing AACS resource if it exists, otherwise create a new one
# name of AACS resource, has to be unique

aacs_name = f"{endpoint_name}-aacs"
available_aacs_locations = ["east us", "west europe"]

# create a new Cognitive Services Account
kind = "ContentSafety"
aacs_sku_name = "S0"
aacs_location = available_aacs_locations[0]


print("Available SKUs:")
aacs_skus = acs_client.resource_skus.list()
print("SKU Name\tSKU Tier\tLocations")
for sku in aacs_skus:
    if sku.kind == "ContentSafety":
        locations = ",".join(sku.locations)
        print(sku.name + "\t" + sku.tier + "\t" + locations)

print(
    f"Choose a new Azure AI Content Safety (AACS) resource in {aacs_location} with SKU {aacs_sku_name}"
)

#### 2.2 Create AACS resource

In [ ]:
parameters = Account(
    sku=Sku(name=aacs_sku_name),
    kind=kind,
    location=aacs_location,
    properties=AccountProperties(
        custom_sub_domain_name=aacs_name, public_network_access="Enabled"
    ),
)


def find_acs(accounts):
    return next(
        x
        for x in accounts
        if x.kind == "ContentSafety"
        and x.location == aacs_location
        and x.sku.name == aacs_sku_name
    )


try:
    # check if AACS exists
    aacs = acs_client.accounts.get(resource_group, aacs_name)
    print(f"Found existing Azure AI content safety (AACS) Account {aacs.name}.")
except:
    try:
        # check if there is an existing AACS resource within same resource group
        aacs = find_acs(acs_client.accounts.list_by_resource_group(resource_group))
        print(
            f"Found existing Azure AI content safety (AACS) Account {aacs.name} in resource group {resource_group}."
        )
    except:
        print(f"Creating Azure AI content safety (AACS) Account {aacs_name}.")
        acs_client.accounts.begin_create(resource_group, aacs_name, parameters).wait()
        print("Resource created.")
        aacs = acs_client.accounts.get(resource_group, aacs_name)


aacs_endpoint = aacs.properties.endpoint
aacs_resource_id = aacs.id
print(f"AACS endpoint is {aacs_endpoint}")
print(f"AACS ResourceId is {aacs_resource_id}")

aacs_access_key = acs_client.accounts.list_keys(
    resource_group_name=resource_group, account_name=aacs.name
).key1

### 3. Create AACS enabled text-to-image batch endpoint

#### 3.1 Check if text-to-image model is available in the AML registry

Browse models in the Model Catalog in the AzureML Studio, filtering by the text-to-image task. In this example, we use the runwayml-stable-diffusion-v1-5 model. If you have opened this notebook for a different model, replace the model name accordingly. This is a pre-trained model.

In [ ]:
try:
    # Name of the text-to-image model to be deployed
    model_name = "runwayml-stable-diffusion-v1-5"
    model = reg_client.models.get(model_name, label="latest")
    print(
        f"Using model name: {model.name}, version: {model.version}, id: {model.id} for inference."
    )
except:
    raise Exception(
        f"No model named {model_name} found in registry. "
        "Please check model name in Azure model catalog."
    )

##### 3.2 Create environment for text-to-image endpoint


In [ ]:
try:
    env = ml_client.environments.get(environment_name, label="latest")
    print("---Environment already exists---")
except:
    print("---Creating environment---")
    env = Environment(
        name=environment_name,
        build=BuildContext(path="./scoring-files/docker_env"),
    )
    ml_client.environments.create_or_update(env)
    env = ml_client.environments.get(environment_name, label="latest")
    print("---Please use link below to check build status---")


display(
    HTML(
        f"""
             <a href="https://ml.azure.com/environments/{environment_name}/version/{env.version}?wsid=/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace}">
                Click here to check env build status in AML studio
             </a>
             """
    )
)

##### 3.3 Create compute cluster to run batch job on

Use the model card from the AzureML system registry to check the minimum required inferencing SKU, referenced as size below. If you already have a sufficient compute cluster that you wish to use, you can simply define the name in `compute_name` in the following code block. Otherwise, the below snippet will create a new compute cluster.

In [ ]:
from azure.ai.ml.constants import AssetTypes, BatchDeploymentOutputAction

if not any(filter(lambda m: m.name == compute_name, ml_client.compute.list())):
    compute_cluster = AmlCompute(
        name=compute_name,
        size=sku_name,
        min_instances=0,
        max_instances=2,
    )
    ml_client.compute.begin_create_or_update(compute_cluster).result()

##### 3.4 Deploy the model to a batch endpoint

Batch endpoints are endpoints that are used to do batch inferencing on large volumes of data over a period of time. The endpoints receive pointers to data and run jobs asynchronously to process the data in parallel on compute clusters. Batch endpoints store outputs to a data store for further analysis. For more information on batch endpoints and deployments, see <a href="https://learn.microsoft.com/en-us/azure/machine-learning/concept-endpoints?view=azureml-api-2#what-are-batch-endpoints" target="_blank"> What are batch endpoints?</a> In this sub-section, we will cover the following items:

* Create a batch endpoint.
* Create a batch deployment.
* Set the deployment as default. Doing so allows invoking the endpoint without specifying the deployment's name.

##### Create a batch endpoint

In [ ]:
from azure.ai.ml.entities import BatchEndpoint

# Check if the endpoint already exists in the workspace
try:
    endpoint = ml_client.batch_endpoints.get(endpoint_name)
    print("---Endpoint already exists---")
except:
    # Create an batch endpoint if it doesn't exist

    # Define the endpoint
    endpoint = BatchEndpoint(name=endpoint_name, description="Test endpoint for model")

    # Trigger the endpoint creation
    try:
        ml_client.begin_create_or_update(endpoint).wait()
        print("\n---Endpoint created successfully---\n")
    except Exception as err:
        raise RuntimeError(
            f"Endpoint creation failed. Detailed Response:\n{err}"
        ) from err

##### Deploy text-to-image model
This step may take a few minutes.

__Note__: `mini_batch_size` is the number of CSV files processed by the model in a single mini_batch.

In [ ]:
deployment = ModelBatchDeployment(
    name=deployment_name,
    endpoint_name=endpoint.name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="scoring-files/score",
        scoring_script="score_batch.py",
    ),
    compute=compute_name,
    settings=ModelBatchDeploymentSettings(
        instance_count=1,
        max_concurrency_per_instance=1,
        mini_batch_size=2,
        output_action=BatchDeploymentOutputAction.APPEND_ROW,
        output_file_name="predictions.csv",
        retry_settings=BatchRetrySettings(max_retries=3, timeout=3000),
        logging_level="info",
        environment_variables={
            "CONTENT_SAFETY_ENDPOINT": aacs_endpoint,
            "CONTENT_SAFETY_KEY": aacs_access_key,
        },
    ),
)
# Trigger the deployment creation
try:
    ml_client.begin_create_or_update(deployment).wait()
    print("\n---Deployment created successfully---\n")
except Exception as err:
    raise RuntimeError(
        f"Deployment creation failed. Detailed Response:\n{err}"
    ) from err

##### Update Batch endpoint to set the default deployment

In [ ]:
endpoint = ml_client.batch_endpoints.get(endpoint_name)
endpoint.defaults.deployment_name = deployment.name
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

### 4. Prepare data for inference - using a folder of csv files with text prompts

We provide the text prompts in a csv file starting from the first row of a column named "prompt". The deployment in the Create batch deployment section below takes the argument mini_batch_size, which is the number of CSV files processed by the model in a single mini_batch. To limit the number of prompts processed in each mini_batch we split the dataset into multiple csv files.

In [ ]:
# Read all the csvs in the data folder into a pandas dataframe
import glob
import os
import pandas as pd

# Specify the folder where your CSV files are located
data_path = "scoring-files/text-to-image-batch-data"

# Use glob to get a list of CSV files in the folder
csv_files = glob.glob(os.path.join(data_path, "*.csv"))

# Read all CSV files into a single DataFrame using pd.concat
batch_df = pd.concat((pd.read_csv(file) for file in csv_files), ignore_index=True)

# Now, 'batch_df' contains all the data from the CSV files in the folder
print(batch_df.head())

In [ ]:
from pathlib import Path

# Specify the folder where your CSV files should be saved
processed_dataset_parent_dir = "scoring-files/processed-text-to-image-batch-data"
os.makedirs(processed_dataset_parent_dir, exist_ok=True)
batch_input_file = "batch_input.csv"

# Divide this into files of <x> rows each
batch_size_per_predict = 2
for i in range(0, len(batch_df), batch_size_per_predict):
    j = i + batch_size_per_predict
    batch_df[i:j].to_csv(
        os.path.join(processed_dataset_parent_dir, str(i) + batch_input_file)
    )

# Check out the first and last file name created
input_paths = sorted(Path(processed_dataset_parent_dir).iterdir(), key=os.path.getmtime)
input_files = [os.path.basename(path) for path in input_paths]
print(f"{input_files[0]} to {str(i)}{batch_input_file}.")

Register folder containing csv files in AML as data asset to use in batch job.

In [ ]:
dataset_name = "text-to-image-data"
input = Data(
    name=dataset_name,
    description="A sample of the dataset for image generation for batch deployment, in CSV file format",
    type=AssetTypes.URI_FOLDER,
    path=processed_dataset_parent_dir,
)
ml_client.data.create_or_update(input)

#### 5. Test the endpoint - using csv files

Invoke the batch endpoint with the input parameter pointing to the directory containing one or more csv files containing the batch inference input. This creates a pipeline job using the default deployment in the endpoint. Wait for the job to complete.

__Note__: If job failed with Out of Memory Error then please try splitting your input into smaller csv files or decreasing mini_batch_size for the deployment.

In [ ]:
input_data = ml_client.data.get(dataset_name, label="latest")
input = Input(type=AssetTypes.URI_FOLDER, path=input_data.id)
job = ml_client.batch_endpoints.invoke(endpoint_name=endpoint.name, input=input)

#### 5.1 Get the details of the invoked job
Let us get details and logs of the invoked job.

In [ ]:
ml_client.jobs.get(job.name)

We can wait for the job to finish using the following code.

In [ ]:
ml_client.jobs.stream(job.name)

__Note__: If the job failed with error Assertion Error (The actual length exceeded max length 100 MB) then please consider dividing input csv file into multiple csv files.

#### 5.2 Download the results

The deployment creates a child job that executes the scoring. We can get the details of it using the following code.

In [ ]:
scoring_job = list(ml_client.jobs.list(parent_job_name=job.name))[0]

In [ ]:
print("Job name:", scoring_job.name)
print("Job status:", scoring_job.status)
print(
    "Job duration:",
    scoring_job.creation_context.last_modified_at
    - scoring_job.creation_context.created_at,
)

The outputs generated by the deployment job will be placed in an output named `named-outputs/score`.

In [ ]:
ml_client.jobs.download(name=scoring_job.name, download_path=".", output_name="score")

In [ ]:
# Load the batch predictions file with no headers into a dataframe and set your column names
import os
import pandas as pd

predictions_file = os.path.join("named-outputs", "score", "predictions.csv")

score_df = pd.read_csv(
    predictions_file,
    header=None,
    names=[
        "row_number_per_file",
        "prompt",
        "generated_image",
        "nsfw_content_detected",
        "file_name",
    ],
)
score_df

### 6 Clean up resources - delete the endpoint
Batch endpoints use compute resources only when jobs are submitted. You can keep the batch endpoint for your reference without worrying about compute bills, or choose to delete the endpoint. If you created your compute cluster to have zero minimum instances and scale down soon after being idle, you won't be charged for an unused compute.

In [ ]:
ml_client.batch_endpoints.begin_delete(endpoint.name).result()